## Imports, Read data, and Generate Model

In [22]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.externals import joblib
import requests, json

df = pd.read_csv("./SalaryData.csv")

train_set, test_set = train_test_split(df, test_size=0.2, random_state=42)

df_copy = train_set.copy()

test_set_full = test_set.copy()
test_set = test_set.drop(["Salary"], axis=1)

train_labels = df_copy["Salary"]

train_set_full = train_set.copy()
train_set = train_set.drop(["Salary"], axis=1)

lin_reg = LinearRegression()

lin_reg.fit(train_set, train_labels)

salary_pred = lin_reg.predict(test_set)

salary_pred

array([ 115790.21011287,   71498.27809463,  102596.86866063,
         75267.80422384,   55477.79204548,   60189.69970699])

## Run Model API

### Persist Model and Data

#### Python Pickle

In [6]:
import pickle

with open("python_lin_reg_model.pkl", "wb") as file_handler:
    pickle.dump(lin_reg, file_handler)
    
with open("python_lin_reg_model.pkl", "rb") as file_handler:
    loaded_pickle = pickle.load(file_handler)
    
loaded_pickle

LinearRegression(copy_X=True, fit_intercept=True, n_jobs=1, normalize=False)

#### `scikit-learn`'s `joblib`

In [25]:
BASE_URL = "http://localhost:5000"

joblib.dump(lin_reg, "linear_regression_model.pkl")

joblib.dump(train_set, "training_data.pkl")
joblib.dump(train_labels, "training_labels.pkl")

['training_labels.pkl']

### Predict API

In [20]:
years_exp = {"yearsOfExperience": 8}

response = requests.post("{}/predict".format(BASE_URL), json = years_exp)

response.json()

[100712.10559602463]

In [13]:
df_copy.query('YearsExperience > 7 and YearsExperience <= 8')

,YearsExperience,Salary
22,7.9,101302.0
21,7.1,98273.0


### Retrain API

In [14]:
data = json.dumps([{"YearsExperience": 12,"Salary": 140000}, 
                   {"YearsExperience": 12.1,"Salary": 142000}])

data

'[{"YearsExperience": 12, "Salary": 140000}, {"YearsExperience": 12.1, "Salary": 142000}]'

In [16]:
response = requests.post("{}/retrain".format(BASE_URL), json = data)

response.json()

'Retrained model successfully.'

In [17]:
response = requests.post("{}/predict".format(BASE_URL), json = years_exp)

response.json()

[101090.28347749889]

### Current Details API

In [26]:
response = requests.get("{}/currentDetails".format(BASE_URL))

response.json()

{'coefficients': [9423.815323030976],
 'intercepts': 25321.583011776813,
 'score': 0.9645401573418146}